In [2]:
!pip install kagglehub

import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import kagglehub
import os
import re
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler


warnings.simplefilter(action='ignore', category=FutureWarning)

path = kagglehub.dataset_download("rajugc/laptop-selection-dataset")


print(os.listdir(path))


data = pd.read_csv(f"{path}/laptops.csv", encoding='ISO-8859-1')
data.head()

100%|██████████| 39.1k/39.1k [00:00<00:00, 34.6MB/s]

Extracting files...
['laptops.csv']


,Unnamed: 0,img_link,name,price(in Rs.),processor,ram,os,storage,display(in inch),rating,no_of_ratings,no_of_reviews
0,0,https://rukminim1.flixcart.com/image/312/312/x...,Lenovo Intel Core i5 11th Gen,62990,Intel Core i5 Processor (11th Gen),16 GB DDR4 RAM,Windows 11 Operating System,512 GB SSD,15.6,4.5,14.0,1.0
1,1,https://rukminim1.flixcart.com/image/312/312/x...,Lenovo V15 G2 Core i3 11th Gen,37500,Intel Core i3 Processor (11th Gen),8 GB DDR4 RAM,64 bit Windows 11 Operating System,1 TB HDD|256 GB SSD,15.6,4.4,53.0,3.0
2,2,https://rukminim1.flixcart.com/image/312/312/l...,ASUS TUF Gaming F15 Core i5 10th Gen,49990,Intel Core i5 Processor (10th Gen),8 GB DDR4 RAM,Windows 11 Operating System,512 GB SSD,15.6,4.4,4733.0,463.0
3,3,https://rukminim1.flixcart.com/image/312/312/x...,ASUS VivoBook 15 (2022) Core i3 10th Gen,33990,Intel Core i3 Processor (10th Gen),8 GB DDR4 RAM,64 bit Windows 11 Operating System,512 GB SSD,15.6,4.3,10406.0,1040.0
4,4,https://rukminim1.flixcart.com/image/312/312/x...,Lenovo Athlon Dual Core,18990,AMD Athlon Dual Core Processor,4 GB DDR4 RAM,DOS Operating System,256 GB SSD,14.0,3.8,18.0,3.0


In [3]:
n_laptop = len(data['name'].unique())
n_ratings = len(data['no_of_ratings'].unique())
n_ram= len(data['ram'].unique())
n_processor= len(data['processor'].unique())
n_os= len(data['os'].unique())
n_storage= len(data['storage'].unique())
n_display= len(data['display(in inch)'].unique())


print(f"Number of laptops: {n_laptop}")
print(f"Number of ratings: {n_ratings}")
print(f"Number of unique ram: {n_ram}")
print(f"Number of unique processors: {n_processor}")
print(f"Number of unique os: {n_os}")
print(f"Number of unique storage: {n_storage}")
print(f"Number of unique display: {n_display}")





Number of laptops: 506
Number of ratings: 274
Number of unique ram: 22
Number of unique processors: 59
Number of unique os: 11
Number of unique storage: 16
Number of unique display: 21


In [4]:
# Extract CPU speed from processor string
def extract_cpu_speed(processor):
    match = re.search(r'(\d+(\.\d+)?)\s*GHz', str(processor))
    return float(match.group(1)) if match else None

# Apply function
data['cpu_speed'] = data['processor'].apply(extract_cpu_speed)
data[['processor', 'cpu_speed']].head()


,processor,cpu_speed
0,Intel Core i5 Processor (11th Gen),None
1,Intel Core i3 Processor (11th Gen),None
2,Intel Core i5 Processor (10th Gen),None
3,Intel Core i3 Processor (10th Gen),None
4,AMD Athlon Dual Core Processor,None


In [5]:
def extract_numbers(text):
    match = re.search(r'\d+', str(text))
    return int(match.group()) if match else None

# Apply to relevant columns
data['ram'] = data['ram'].apply(extract_numbers)
data['storage'] = data['storage'].apply(extract_numbers)

# Convert 'display(in inch)' column to numeric
data['display(in inch)'] = pd.to_numeric(data['display(in inch)'], errors='coerce')

# Convert 'price(in Rs.)' to numeric
data['price(in Rs.)'] = pd.to_numeric(data['price(in Rs.)'], errors='coerce')


In [6]:
def extract_cpu_speed(processor):
    if pd.isna(processor):
        return None  # Handle missing values
    match = re.search(r'(\d+(\.\d+)?)\s*GHz', str(processor))
    return float(match.group(1)) if match else None

# Apply extraction to create 'cpu_speed' column
data['cpu_speed'] = data['processor'].apply(extract_cpu_speed)

# Check if the column was created correctly
print(data[['processor', 'cpu_speed']].head())  # Debugging step

                            processor cpu_speed
0  Intel Core i5 Processor (11th Gen)      None
1  Intel Core i3 Processor (11th Gen)      None
2  Intel Core i5 Processor (10th Gen)      None
3  Intel Core i3 Processor (10th Gen)      None
4      AMD Athlon Dual Core Processor      None


In [7]:
missing_cols = [col for col in ['price(in Rs.)', 'ram', 'storage', 'cpu_speed', 'display(in inch)'] if col not in data.columns]
if missing_cols:
    print(f"Missing columns: {missing_cols}")
else:
    scaler = MinMaxScaler()
    scaled_features = ['price(in Rs.)', 'ram', 'storage', 'cpu_speed', 'display(in inch)']
    data[scaled_features] = scaler.fit_transform(data[scaled_features])


/usr/local/lib/python3.11/dist-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/usr/local/lib/python3.11/dist-packages/sklearn/utils/_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))


In [8]:
scaler = MinMaxScaler()
scaled_features = ['price(in Rs.)', 'ram', 'storage', 'cpu_speed', 'display(in inch)']
data[scaled_features] = scaler.fit_transform(data[scaled_features])


/usr/local/lib/python3.11/dist-packages/sklearn/utils/_array_api.py:776: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/usr/local/lib/python3.11/dist-packages/sklearn/utils/_array_api.py:793: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))


In [9]:
# Assign unique user and item IDs
data['user_id'] = range(len(data))
data['item_id'] = range(len(data))

# Compute rating proxy
data['rating_proxy'] = data[scaled_features].mean(axis=1)


In [10]:
class MatrixFactorization(nn.Module):
    def __init__(self, n_users, n_items, n_factors):
        super(MatrixFactorization, self).__init__()
        self.user_emb = nn.Embedding(n_users, n_factors)
        self.item_emb = nn.Embedding(n_items, n_factors)
        self.bias_user = nn.Embedding(n_users, 1)
        self.bias_item = nn.Embedding(n_items, 1)

    def forward(self, x):
        users = x[:, 0]
        items = x[:, 1]
        pred = (self.user_emb(users) * self.item_emb(items)).sum(1)
        pred += self.bias_user(users).squeeze() + self.bias_item(items).squeeze()
        return pred


In [11]:
# Initialize the model
n_users, n_items = len(data), len(data)
n_factors = 20
model = MatrixFactorization(n_users, n_items, n_factors)
optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.MSELoss()

# Prepare training data
training_data = torch.tensor(data[['user_id', 'item_id', 'rating_proxy']].values, dtype=torch.float32)

# Train the model
for epoch in range(10):
    optimizer.zero_grad()
    predictions = model(training_data[:, :2].long())
    loss = loss_fn(predictions, training_data[:, 2])
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")


Epoch 1, Loss: 21.715988159179688
Epoch 2, Loss: 19.27412986755371
Epoch 3, Loss: 17.061203002929688
Epoch 4, Loss: 15.066325187683105
Epoch 5, Loss: 13.273467063903809
Epoch 6, Loss: 11.666478157043457
Epoch 7, Loss: 10.231879234313965
Epoch 8, Loss: 8.95703411102295
Epoch 9, Loss: 7.828954696655273
Epoch 10, Loss: 6.834188461303711


In [12]:
def recommend_laptops(user_preferences, data, top_n=5):
    # Check if user preferences contain NaN values
    if any(pd.isna(value) for value in user_preferences.values()):
        print("User preferences contain NaNs:")
        print(user_preferences)
        return None  # or you can return an empty list/dataframe as a fallback

    # Check if the dataset contains NaN values
    if data.isnull().any().any():
        print("Data contains NaNs, here are the columns with NaNs:")
        print(data.isnull().sum())
        return None  # or handle the NaNs by filling/dropping them

    # Continue with the recommendation process if no NaN values are found
    user_df = pd.DataFrame([user_preferences])
    user_df = scaler.transform(user_df)  # Scaling the preferences

    # Compute cosine similarity and other steps
    cosine_similarities = cosine_similarity(user_df, data[scaled_features])
    top_indices_cosine = np.argsort(cosine_similarities[0])[::-1][:top_n * 2]

    # Matrix Factorization Predictions
    user_id = 0  # Assuming a single user
    item_ids = data.iloc[top_indices_cosine]['item_id'].values
    user_tensor = torch.tensor([user_id] * len(item_ids), dtype=torch.long)
    item_tensor = torch.tensor(item_ids, dtype=torch.long)
    predictions_mf = model(torch.stack([user_tensor, item_tensor], dim=1)).detach().numpy()
    sorted_indices_mf = np.argsort(predictions_mf)[::-1][:top_n]
    top_indices_mf = item_ids[sorted_indices_mf]

    # Combine recommendations
    recommended_indices = np.intersect1d(top_indices_cosine, top_indices_mf)
    if len(recommended_indices) < top_n:
        remaining = top_n - len(recommended_indices)
        additional_indices = np.setdiff1d(top_indices_cosine, recommended_indices)[:remaining]
        recommended_indices = np.concatenate([recommended_indices, additional_indices])

    recommended_laptops = data.iloc[recommended_indices][['name', 'processor', 'ram', 'storage', 'display(in inch)', 'price(in Rs.)']]
    return recommended_laptops


In [13]:
print(data.columns)

# Fill missing values in specific columns (adjust column names as needed)
data['cpu_speed'].fillna(data['cpu_speed'].median(), inplace=True)

# Fill missing numeric values with median for specific columns
numeric_cols = ['price(in Rs.)', 'ram', 'storage', 'cpu_speed', 'display(in inch)']
data[numeric_cols] = data[numeric_cols].fillna(data[numeric_cols].median())

# Check for missing values
print(data.isnull().sum())  # This should output all zeros if there are no missing values

# Remove rows with any NaN values
data_clean = data.dropna()

# Check if there are still any NaN values
if data_clean.isnull().any().any():
    print("Data still contains NaN values.")
    # Display which columns have NaN values
    print(data_clean.isnull().sum())
else:
    print("No NaN values found in cleaned data.")

# Check specific columns for missing values
columns_to_check = ['price(in Rs.)', 'ram', 'storage', 'cpu_speed', 'display(in inch)']
for col in columns_to_check:
    if col in data_clean.columns:
        print(f"NaN values in {col}: {data_clean[col].isnull().sum()}")
    else:
        print(f"Column {col} does not exist in the DataFrame.")

# Convert to numpy array to check for NaN
# Extract only numeric columns
numeric_data = data_clean.select_dtypes(include=[np.number])

# Convert to numpy array
data_matrix = numeric_data.to_numpy()

# Check for NaN values in the numeric matrix
if np.isnan(data_matrix).any():
    print("Matrix contains NaN values.")
else:
    print("No NaN values in matrix.")

Index(['Unnamed: 0', 'img_link', 'name', 'price(in Rs.)', 'processor', 'ram',
       'os', 'storage', 'display(in inch)', 'rating', 'no_of_ratings',
       'no_of_reviews', 'cpu_speed', 'user_id', 'item_id', 'rating_proxy'],
      dtype='object')
Unnamed: 0            0
img_link              0
name                  0
price(in Rs.)         0
processor             0
ram                   0
os                    0
storage               0
display(in inch)      0
rating              296
no_of_ratings       296
no_of_reviews       296
cpu_speed           984
user_id               0
item_id               0
rating_proxy          0
dtype: int64
No NaN values found in cleaned data.
NaN values in price(in Rs.): 0
NaN values in ram: 0
NaN values in storage: 0
NaN values in cpu_speed: 0
NaN values in display(in inch): 0
No NaN values in matrix.


In [14]:
# Check the data types to identify any non-numeric columns
print(data_clean.dtypes)

# Check if any of the columns are object types
object_columns = data_clean.select_dtypes(include='object').columns
print(f"Object columns: {object_columns}")

# If any object columns have NaNs, handle them by filling or dropping
data_clean[object_columns] = data_clean[object_columns].fillna('Unknown')


Unnamed: 0            int64
img_link             object
name                 object
price(in Rs.)       float64
processor            object
ram                 float64
os                   object
storage             float64
display(in inch)    float64
rating              float64
no_of_ratings       float64
no_of_reviews       float64
cpu_speed           float64
user_id               int64
item_id               int64
rating_proxy        float64
dtype: object
Object columns: Index(['img_link', 'name', 'processor', 'os'], dtype='object')


In [15]:
# Drop object columns (if not required for the recommendation)
data_clean_numeric = data_clean.drop(columns=['img_link', 'name', 'processor', 'os'])

# Check if there are any NaN values left
print(data_clean_numeric.isnull().sum())


Unnamed: 0          0
price(in Rs.)       0
ram                 0
storage             0
display(in inch)    0
rating              0
no_of_ratings       0
no_of_reviews       0
cpu_speed           0
user_id             0
item_id             0
rating_proxy        0
dtype: int64


In [33]:
# Verify the number of columns
print(f"Number of columns in data_numeric: {data_numeric.shape[1]}")
print(f"Length of scaled_features: {len(scaled_features)}")

# If they don't match, update scaled_features to match the number of columns in data_numeric
scaled_features = list(data_clean_numeric.columns)  # For example, take column names from the original dataset

# Then, you can proceed with creating the DataFrame with matching columns
data_numeric_df = pd.DataFrame(data_numeric, columns=scaled_features)

# Now check for NaNs and proceed
if data_numeric_df.isnull().any().any():
    print("Data contains NaNs. Let's handle them.")
    data_numeric_df = data_numeric_df.fillna(data_numeric_df.median())  # Handle NaNs if needed
else:
    print("No NaN values found in the dataset.")

# Now proceed with the recommendation function
recommendations = recommend_laptops(user_preferences_scaled, data_numeric_df)
print(recommendations)


Number of columns in data_numeric: 12
Length of scaled_features: 5
No NaN values found in the dataset.


NameError: name 'user_preferences_scaled' is not defined